# Zipline Coding Exercises

In this notebook you will create your own pipeline and create custom factors and filters for it. 

### Install Packages

In [1]:
import sys
!{sys.executable} -m pip install -r requirements.txt

    100% |████████████████████████████████| 2.5MB 6.8MB/s eta 0:00:01
    100% |████████████████████████████████| 92kB 16.8MB/s ta 0:00:01
    100% |████████████████████████████████| 26.3MB 1.3MB/s eta 0:00:01  2% |▉                               | 675kB 22.1MB/s eta 0:00:02    7% |██▎                             | 1.9MB 22.8MB/s eta 0:00:02    29% |█████████▌                      | 7.8MB 25.0MB/s eta 0:00:01    34% |███████████                     | 9.1MB 23.8MB/s eta 0:00:01    43% |██████████████                  | 11.5MB 25.1MB/s eta 0:00:01    62% |████████████████████▏           | 16.5MB 28.8MB/s eta 0:00:01    67% |█████████████████████▊          | 17.8MB 31.5MB/s eta 0:00:01    89% |████████████████████████████▌   | 23.4MB 24.4MB/s eta 0:00:01    93% |██████████████████████████████  | 24.6MB 23.8MB/s eta 0:00:01    98% |███████████████████████████████▍| 25.7MB 25.2MB/s eta 0:00:01
    100% |████████████████████████████████| 112kB 19.7MB/s ta 0:00:01
    100% |██████████████████

# Load Data

In [5]:
import os

from zipline.data import bundles
from zipline.data.bundles.csvdir import csvdir_equities


# Specify the bundle name
bundle_name = 'm4-quiz-eod-quotemedia'

# Create an ingest function 
ingest_func = csvdir_equities(['daily'], bundle_name)

# Register the data bundle and its ingest function
bundles.register(bundle_name, ingest_func);

# Set environment variable 'ZIPLINE_ROOT' to the path where the most recent data is located
os.environ['ZIPLINE_ROOT'] = os.path.join(os.getcwd(), '..', '..', 'data', 'module_4_quizzes_eod')

# Load the data bundle
bundle_data = bundles.load(bundle_name)

# TODO: Build an Empty Pipeline with a Screen

In the code below create an empty with a screen that filters the pipeline output for stocks with a 60-day average dollar volume greater than \$50,000.

In [6]:
# import resources
from zipline.pipeline import Pipeline
from zipline.pipeline.factors import AverageDollarVolume

# Create a screen for our Pipeline
universe = AverageDollarVolume(window_length = 60) > 50000

# Create an empty Pipeline with the given screen
pipeline = Pipeline(screen = universe)

# TODO: Create a Pipeline Engine

In the code below, create a data loader that loads our data bundle using Zipline's `USEquityPricingLoader`. Then create a function to be passed to the `get_loader` parameter in the pipeline engine. Using the NYSE trading calendar create a pipeline engine using Zipline's `SimplePipelineEngine`

In [8]:
# import resources
from zipline.pipeline.loaders import USEquityPricingLoader
from zipline.utils.calendars import get_calendar
from zipline.pipeline.data import USEquityPricing
from zipline.pipeline.engine import SimplePipelineEngine

# Set the dataloader
pricing_loader = USEquityPricingLoader(bundle_data.equity_daily_bar_reader, bundle_data.adjustment_reader)

# Define the function for the get_loader parameter
def choose_loader(column):
    if column not in USEquityPricing.columns:
        raise Exception('Column not in USEquityPricing')
    return pricing_loader


# Set the trading calendar
trading_calendar = get_calendar('NYSE')

# Create a Pipeline engine
engine = SimplePipelineEngine(get_loader = choose_loader,
                              calendar = trading_calendar.all_sessions,
                              asset_finder = bundle_data.asset_finder)

# TODO: Create Factors and Filters For The Pipeline

### Factors

In the code below, create two factors:

1. **Custom Factor**: Create a custom factor, `percent_difference`,  by combining a 30-day average closing price factor and a 60-average closing price factor. The `percent_difference` factor will calculate the difference between the 30-day and 60-average closing price factors, and will then normalize this difference by the 60-average closing price factor.


2. **Daily Returns Factor**: Create a factor that returns the daily percent change in closing price. To do this, use Zipline's built-in [DailyReturns](https://www.zipline.io/appendix.html#zipline.pipeline.factors.DailyReturns) factor.

Both factors should use the closing prices in the `USEquityPricing`  dataset as inputs. 




### Filters

In the code below, you will create a **Custom Filter**, `tradable_asset`, that returns `True` for the top 20 securities of average dollar volume in a 30 day-window that have a latest closing price above \$30. To do this, you will create two filters first and them combine them. For the first filter, you can use Zipline's built-in factor `AverageDollarVolume()` and its method `.top()` to select the top 20 assets of average dollar volume in a 30 day-window. To get the latest close price of the `USEquityPricing` dataset you can use `USEquityPricing.close.latest`. Finally you can combine these two filters by using the logical operator `&`. 

In [10]:
# import resources
from zipline.pipeline.factors import SimpleMovingAverage
from zipline.pipeline.factors import DailyReturns
from zipline.pipeline.factors import AverageDollarVolume

# Create the 30-day average closing price factor
mean_close_30 = SimpleMovingAverage(inputs = [USEquityPricing.close], window_length = 30)

# Create the 60-day average closing price factor
mean_close_60 = SimpleMovingAverage(inputs = [USEquityPricing.close], window_length = 60)

# Create the custom factor
percent_difference = (mean_close_30 - mean_close_60)/mean_close_60

# Create the daily returns factor
daily_ret = DailyReturns(inputs = [USEquityPricing.close])

# Create a filter for the top 20 securities of average dollar volume in a 30 day-window
top_20 = AverageDollarVolume(window_length = 30).top(20)

# Create a filter for the latest closing price above $30
above_30 = USEquityPricing.close.latest > 30

# Create a custum filter
tradable_asset = top_20 & above_30

# TODO: Add Factors and Filters To The Pipeline

In the code below, add the factors and filters you created above to the pipeline

In [11]:
# Add the custom factor to the pipeline
pipeline.add(percent_difference, 'Percent Difference')

# Add the daily returns factor to the pipeline
pipeline.add(daily_ret, 'Daily Return')

# Add the custom filter to the pipeline
pipeline.add(tradable_asset, 'Tradable')

# TODO: Run The Pipeline

In the code below, run the pipeline for the dates given

In [14]:
import pandas as pd

# Set starting and end dates
start_date = pd.Timestamp('2014-01-06', tz='utc')
end_date = pd.Timestamp('2016-01-05', tz='utc')

# Run our pipeline for the given start and end dates
output = engine.run_pipeline(pipeline, start_date, end_date)

# Display the pipeline output
output.head(100)

Daily Return  \
2014-01-06 00:00:00+00:00 Equity(0 [A])           0.006951   
                          Equity(1 [AAL])         0.046523   
                          Equity(2 [AAP])         0.028611   
                          Equity(3 [AAPL])       -0.021972   
                          Equity(4 [ABBV])        0.006147   
                          Equity(5 [ABC])         0.000704   
                          Equity(6 [ABT])         0.010701   
                          Equity(7 [ACN])         0.003335   
                          Equity(8 [ADBE])       -0.002193   
                          Equity(9 [ADI])         0.006708   
                          Equity(10 [ADM])        0.004652   
                          Equity(11 [ADP])        0.010890   
                          Equity(12 [ADS])        0.005186   
                          Equity(13 [ADSK])      -0.007107   
                          Equity(14 [AEE])       -0.003676   
                          Equity(15 [AEP])       -0.000435   
                          Equity(16 [AES])       -0.004972   
                          Equity(17 [AET])        0.004288   
                          Equity(18 [AFL])        0.002872   
                          Equity(19 [AGN])       -0.000176   
                          Equity(20 [AIG])        0.004134   
                          Equity(21 [AIV])        0.000000   
                          Equity(22 [AIZ])        0.006133   
                          Equity(23 [AJG])        0.000223   
                          Equity(24 [AKAM])      -0.001719   
                          Equity(25 [ALB])        0.001757   
                          Equity(26 [ALGN])       0.003687   
                          Equity(27 [ALK])        0.018311   
                          Equity(28 [ALL])       -0.004098   
                          Equity(29 [ALLE])       0.007102   
...                                                    ...   
                          Equity(70 [BMY])        0.011103   
                          Equity(71 [BSX])        0.001679   
                          Equity(72 [BWA])       -0.000922   
                          Equity(73 [BXP])        0.011957   
                          Equity(74 [C])          0.021606   
                          Equity(75 [CA])         0.020931   
                          Equity(76 [CAG])       -0.002393   
                          Equity(77 [CAH])        0.009987   
                          Equity(78 [CAT])       -0.000552   
                          Equity(79 [CB])        -0.002558   
                          Equity(80 [CBG])        0.002278   
                          Equity(81 [CBOE])       0.013336   
                          Equity(82 [CBS])       -0.001731   
                          Equity(83 [CCI])       -0.005446   
                          Equity(84 [CCL])        0.001015   
                          Equity(85 [CELG])      -0.012388   
                          Equity(86 [CERN])      -0.006719   
                          Equity(87 [CF])        -0.006883   
                          Equity(89 [CHD])        0.000164   
                          Equity(90 [CHK])       -0.007492   
                          Equity(91 [CHRW])      -0.009333   
                          Equity(92 [CHTR])      -0.010278   
                          Equity(93 [CI])        -0.005446   
                          Equity(94 [CINF])      -0.006205   
                          Equity(95 [CL])         0.001480   
                          Equity(96 [CLX])        0.003622   
                          Equity(97 [CMA])        0.004900   
                          Equity(98 [CMCSA])     -0.007358   
                          Equity(99 [CME])        0.015907   
                          Equity(100 [CMG])       0.015056   

                                              Percent Difference  Tradable  
2014-01-06 00:00:00+00:00 Equity(0 [A])                 0.025939     False  
                          Equity(1 [AAL])      

## Solution

[Solution notebook](zipline_coding_exercises_solution.ipynb)